Diese Version ist durchschnittlich 16 % schneller

In [1]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
import json
import time

from interessant import * # Bei Änderungen Kernel neu starten

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34, OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[0] 
filename = interessant[key]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Einfach 4473900_5335875.copc.laz


In [3]:
thresh = 8  # z.B. 5 oder 8
majority_tresh  = 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis
ground_percentile = 10 # 10 in Oude E.
top_percentile = 99.5 # 90 in Oude E.

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.

with_normals = False

Voxel size: 0.8333333333333334


In [4]:
import subprocess
# subprocess.Popen(["pyvistaviewer", filename])

In [5]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]
points.dtype # Column names and types

dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('Synthetic', 'u1'), ('KeyPoint', 'u1'), ('Withheld', 'u1'), ('Overlap', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('ScanChannel', 'u1'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])

In [6]:
points['Classification'] = 0

In [7]:
points.shape 

(1803127,)

In [8]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
rgb = np.vstack((points['Red'], points['Green'], points['Blue'])).transpose() / 65535.0

intensity = points['Intensity']
intensity_normalized = (intensity - intensity.min()) / (intensity.max() - intensity.min())
colormap = plt.get_cmap("viridis")
intensity_colors = colormap(intensity_normalized)
intensity_colors = intensity_colors[:, :3]

# Offset entfernen (aber gerundet, damit Kachelgrenzen ganze Zahlen bleiben)
offset = xyz.mean(axis=0).round() 
xyz -= offset

In [9]:
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([11.99997543, 11.99998693, 13.12408   ]),
 array([-13.00002457, -13.00001307,  -3.99652   ]))

In [10]:
RAIL = 20

In [11]:
voxels = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [12]:
class GeometryConstraints:


    def __init__(self, voxel_size, voxel_minimum_points, empty_space_thresh, ground_percentile, top_percentile, majority_thresh):
        self.voxel_size = voxel_size
        self.voxel_minimum_points = voxel_minimum_points
        self.empty_space_thresh = empty_space_thresh
        self.ground_percentile = ground_percentile
        self.top_percentile = top_percentile
        self.majority_thresh = majority_thresh

        self.foo = 0.3

    def check(self, z_values):  
        # Threshold on number of points in voxel
        if len(z_values) < self.voxel_minimum_points:
            return None

        ground_level = np.percentile(z_values, self.ground_percentile) 
        # 10% Percentile
        # Check that there are almost no points 0.5 to 4.5 m above the ground
        # But allow for some noise

        count = ((z_values > ground_level + self.foo) & (z_values < ground_level + 4.5)).sum()

        if count <= self.empty_space_thresh:
            # Look for points within 0.5 m above ground and get 
            # 98 percentile (Oude E.) or better 99.5 percentile
            mask = (z_values > ground_level) & (z_values < ground_level + self.foo)
            try:
                candidates_top = np.percentile(z_values[mask], self.top_percentile)
            except IndexError:
                # Fails if there are no points in the masked array
                return None

            # Oude Elberink require the height difference > 0.1 m
            # And mark only the points 10 cm below the top as rail point candidates
            if candidates_top - ground_level > 0.1:
                mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

                # Also make sure these are only a minority of the points (otherwise it's a slope)
                if mask.sum() < self.majority_thresh * len(z_values):  # z.B. 0.3
                    return mask
        return None

In [13]:
constraints = GeometryConstraints(voxel_size, minimum_points, thresh, ground_percentile, top_percentile, majority_tresh)

In [14]:
v_max = voxels.max(axis=0)

In [15]:
z = xyz[:, 2]

Bei "Einfach" 3 s

In [16]:
start = time.time()

for i in range (v_max[0] + 1):
    for j in range(v_max[1] + 1):
        mask = (voxels[:,0] == i) & (voxels[:,1] == j)
        z_values = z[mask]
        mask2 = constraints.check(z_values)
        if mask2 is not None:
            indices = np.where(mask)[0]
            points['Classification'][indices[mask2]] = RAIL

print("Time:", time.time() - start)

Time: 4.256779670715332


In [17]:
candidates = points[points["Classification"] == RAIL]

In [18]:
len(candidates)

87092

In [19]:
# Viewsettings mit strg + c kopieren und hier einfügen

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

In [20]:
xyz2 = np.vstack((candidates['X'], candidates['Y'], candidates['Z'])).transpose()
xyz2 -= offset
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz2)
o3d.visualization.draw_geometries([pcd], front=front, lookat=lookat, up=up, zoom=zoom)